### Scrapping des données :

In [185]:
from selenium import webdriver
import pandas as pd

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [186]:

browser = webdriver.Firefox()
browser.get('https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0')

## Tests :

In [187]:
conteneur_des_resultats = browser.find_element(By.CLASS_NAME,"container-result")# recuprer la div qui contient l'ensemble d'annonces
resultats = conteneur_des_resultats.find_elements(By.XPATH,"./div/a") # Recuperer les liens vers le descriptif de chaque annonce
resultats.pop()
for element in resultats:
    print(element.get_attribute('href'))

https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172982802W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=0
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172631521W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=1
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/173032734W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=2
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/173012464W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=3
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172934112W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConventio

## Changement de page :
dans cette partie on doit sauvegarder a chaque fois la page precedente afin de pouvoir y retourner ou peut etre pas ! a savoir: Les etapes a suivre sont donc les suivantes : 
-   Dans un premier lieu on genere la premiere url en  specifiant le mot cle souhaité qu'on va passer au main (ou pour cette etape on peut directemnt prendre la premiere url en dure comme etant directeemnt la la page de recherche resultatnte dans apec a la saisie de data dans la recherche)
-   Ensuite on recupere le nombre de page de resultats
-   Ensuite à get_url on va parcour toutes ces pages de resultas
-   POur chaque page on recupère la div ayant le nom de class 'container-result' qui contient la liste des dic de chaque annonce
-   On récupere pour chaque annonce le lien de sa description détaillée 
-   On se met sur la page de description on scrappe nos donnnees
-   On parcourt de cette mannière toutes les pages ainsi que toutes les offres 
-   ET voilouuuu

## Changement de page (TODO APRES)

### Ce bout de code est a utilsier pour passer a la page de description de chaque annonce :

In [188]:
# browser.execute_script('arguments[0].target="_self"; arguments[0].click();', "https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172982802W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=0")
script = 'window.location.href = "'+resultats[0].get_attribute('href')+'";'
browser.execute_script(script)

## Tache 1 : scraper la page détaillée de l'annonce :
POur le moment ici comme on ne parcourt pas encore les page il faut decomenter les 2 derniere ligne afin de charger(la premiere annonce par exemple) dans le browser


In [ ]:
# Ne pas executer plusieurs ici on passe sur la page detaillée de la première annonce 
# d'où le zérro ! Donc dans browser on a le contenu de la nouvelle page !
# script = 'window.location.href = "'+resultats[0].get_attribute('href')+'";'
# browser.execute_script(script)

#### Récupèration des informations du réctangle en haut de la page :
    -   Ici je récupère le nom de l'entreprise, type du contrat proposé, departement , date de publication, date d'actualisation 

In [189]:
informations_cles = ['entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire','descriptif_entreprise']

# Initialiser un dictionnaire avec des clés et des valeurs initiales à une liste vide
dictionnaire_annonces ={}
for cle in informations_cles:
    dictionnaire_annonces[cle]=[]
dictionnaire_annonces

{'entreprise': [],
 'type_contrat': [],
 'deparetement': [],
 'date_publication': [],
 'date_mise_a_jour': [],
 'salaire': [],
 'prise_de_poste': [],
 'experience': [],
 'metier': [],
 'statut_du_poste': [],
 'zone_de_deplacement': [],
 'secteur_d_activite': [],
 'Teletravail': [],
 'descriptif_poste': [],
 'profil_recherche': [],
 'langues': [],
 'savoir_etre': [],
 'savoir_faire': [],
 'descriptif_entreprise': []}

In [190]:
informations_cles = ['entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire','descriptif_entreprise']

# Initialiser un dictionnaire avec des clés et des valeurs initiales à une liste vide
dictionnaire_annonces ={}
for cle in informations_cles:
    dictionnaire_annonces[cle]=[]

def click_bouton_Vor_Plus(browser) :
    #cliquer sur voir plus 
    button = browser.find_element(By.XPATH, "//span[text()='Voir plus']")
    button.click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Voir plus']"))).click()

def recuperer_une_offre(browser , dictionnaire_annonces):

    # entreprise , type_contrat , departement
    entete_annonce = browser.find_element(By.CLASS_NAME,"card-offer__text")# recuprer la div du petit réctangle
    informations = entete_annonce.find_elements(By.XPATH,"./ul/li") # Recuperer les liens vers le descriptif de chaque annonce
    for i,info in enumerate(informations):
        cle = informations_cles[i]
        mot = info.text.strip()
        dictionnaire_annonces[cle].append(mot)



    # 'date_publication','date_mise_a_jour'
    date_publication = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[0].text
    date_mise_a_jour = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[1].text
    dictionnaire_annonces['date_publication'].append(date_publication)
    dictionnaire_annonces['date_mise_a_jour'].append(date_mise_a_jour)


    # 'salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail'
    apec_poste_imformations = browser.find_element(By.TAG_NAME,"apec-poste-informations")# recuprer la div du petit réctangle
    liste_informations = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
    salaire = None
    prise_de_poste= None
    experience = None
    metier = None
    statut_du_poste = None
    zone_de_deplacement = None
    secteur_d_activite = None
    Teletravail = None
    for information in liste_informations:
        reponse = information.find_elements(By.XPATH, "./h4[text()='Salaire']/following-sibling::span")
        if reponse :
            salaire = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Prise de poste']/following-sibling::span")
        if reponse :
            prise_de_poste = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Expérience']/following-sibling::span")
        if reponse :
            experience = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Métier']/following-sibling::span")
        if reponse :
            metier = reponse[0].text

        reponse = information.find_elements(By.XPATH, "./h4[text()='Statut du poste']/following-sibling::span")
        if reponse :
            statut_du_poste = reponse[0].text
            
        reponse = information.find_elements(By.XPATH, "./h4[text()='Zone de déplacement']/following-sibling::span")
        if reponse :
            zone_de_deplacement = reponse[0].text

        reponse = information.find_elements(By.XPATH, "./h4[text()='Secteur d’activité du poste']/following-sibling::span")
        if reponse :
            secteur_d_activite = reponse[0].text     
        
        reponse = information.find_elements(By.XPATH, "./h4[text()='Télétravail']/following-sibling::span")
        if reponse :
            Teletravail = reponse[0].text    

    dictionnaire_annonces['salaire'].append(salaire)
    dictionnaire_annonces['prise_de_poste'].append(prise_de_poste)
    dictionnaire_annonces['experience'].append(experience)
    dictionnaire_annonces['metier'].append( metier)
    dictionnaire_annonces['statut_du_poste'].append(statut_du_poste)
    dictionnaire_annonces['zone_de_deplacement'].append(zone_de_deplacement)
    dictionnaire_annonces['secteur_d_activite'].append(secteur_d_activite)
    dictionnaire_annonces['Teletravail'].append(Teletravail)


    #'descriptif_poste' 'profil_recherche' 
    descriptifs = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
    descriptif_poste = None
    descriptif_profil = None
    for desc in descriptifs:
        reponse = desc.find_elements(By.XPATH, "./h4[text()='Descriptif du poste']/following-sibling::p")
        if reponse:
            descriptif_poste=reponse[0].text
        reponse = desc.find_elements(By.XPATH, "./h4[text()='Profil recherché']/following-sibling::p")
        if reponse:
            descriptif_profil=reponse[0].text

    dictionnaire_annonces['descriptif_poste'].append(descriptif_poste)
    dictionnaire_annonces['profil_recherche'].append(descriptif_profil)

    # Langues
    langues_requises = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__language")
    liste_langues_requises= langues_requises.find_elements(By.XPATH, "//h5[text()='Langues']/../following-sibling::*")
    Langues_L = []
    for langue in liste_langues_requises:
        text_lu = langue.text
        if text_lu != "Voir moins" :
            Langues_L.append(langue.text)
    dictionnaire_annonces['langues'].append(Langues_L)


    #savoir-être
    soft_skills = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__knowhow")
    liste_soft_skills = soft_skills.find_elements(By.XPATH, "//h5[text()='Savoir-être']/../following-sibling::*")

    soft_skills_L = []
    for sf in liste_soft_skills:
        text_lu = sf.text
        if text_lu != "Voir moins" :
            soft_skills_L.append(sf.text)
    dictionnaire_annonces['savoir_etre'].append(soft_skills_L)

    #savoir-faire
    hard_skills = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__knowledge")
    liste_hard_skills = hard_skills.find_elements(By.XPATH, "//h5[text()='Savoir-faire']/../following-sibling::*")

    hard_skills_L = []
    for hs in liste_hard_skills:
        #print(hs.get_attribute("innerHTML"))
        text_lu = hs.text
        if text_lu != "Voir moins" :
            hard_skills_L.append(hs.text)
    dictionnaire_annonces['savoir_faire'].append(hard_skills_L)

    #Discption d'entreprise
    descriptif_entreprise = None
    desc_entreprise_element = apec_poste_imformations.find_element(By.XPATH, "//h4[text()='Entreprise']/following-sibling::p")
    if desc_entreprise_element:
            descriptif_entreprise=desc_entreprise_element.text
    dictionnaire_annonces["descriptif_entreprise"].append(descriptif_entreprise)

    
    

def offre_suivante(browser):
    next_page_button = browser.find_element(By.CLASS_NAME,"nextpage")
    next_page_button.click()


NB_JOBS = 900
click_bouton_Vor_Plus(browser)
for i in range(NB_JOBS):
    try:
        print(i)
        recuperer_une_offre(browser ,dictionnaire_annonces)
        
        element = WebDriverWait(browser,10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME,"nextpage"))
        )
        element[0].click()
        
    except:
        print("Plus d'offre suivante, on peut fermer la fenetre")
        #browser.quit()
recuperer_une_offre(browser , dictionnaire_annonces)

    

#dictionnaire_annonces

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [191]:
for cle, liste_dictionnaires in dictionnaire_annonces.items():
    taille_liste = len(liste_dictionnaires)  # Calcul de la taille de la liste
    print(f"La taille de la liste de la clé '{cle}' est : {taille_liste}")

La taille de la liste de la clé 'entreprise' est : 501
La taille de la liste de la clé 'type_contrat' est : 501
La taille de la liste de la clé 'deparetement' est : 501
La taille de la liste de la clé 'date_publication' est : 501
La taille de la liste de la clé 'date_mise_a_jour' est : 501
La taille de la liste de la clé 'salaire' est : 501
La taille de la liste de la clé 'prise_de_poste' est : 501
La taille de la liste de la clé 'experience' est : 501
La taille de la liste de la clé 'metier' est : 501
La taille de la liste de la clé 'statut_du_poste' est : 501
La taille de la liste de la clé 'zone_de_deplacement' est : 501
La taille de la liste de la clé 'secteur_d_activite' est : 501
La taille de la liste de la clé 'Teletravail' est : 501
La taille de la liste de la clé 'descriptif_poste' est : 501
La taille de la liste de la clé 'profil_recherche' est : 501
La taille de la liste de la clé 'langues' est : 501
La taille de la liste de la clé 'savoir_etre' est : 501
La taille de la lis

## Création du dataFrame

In [192]:
import pandas as pd
df = pd.DataFrame(dictionnaire_annonces)
df.tail()

,entreprise,type_contrat,deparetement,date_publication,date_mise_a_jour,salaire,prise_de_poste,experience,metier,statut_du_poste,zone_de_deplacement,secteur_d_activite,Teletravail,descriptif_poste,profil_recherche,langues,savoir_etre,savoir_faire,descriptif_entreprise
496,IT&M STATS,1 \n CDI,Boulogne-Billancourt - 92,Publiée le 17/01/2024,Actualisée le 17/01/2024,A négocier,Dès que possible,Minimum 5 ans,Chief Data Officer,Cadre du secteur privé,Pas de déplacement,CONSEIL EN SYSTÈMES ET LOGICIELS INFORMATIQUES,None,Vos missions principales\nAssurer la qualité d...,Vous êtes diplômé en Informatique et/ou filièr...,[Anglais],"[Capacité d'analyse, Esprit d'équipe, Forte ca...","[Data Management, SAS, Archivage, Base de donn...",IT&M Stats intervient dans le domaine des stat...
497,Groupe ADP - Aéroport de Paris,1 \n CDI,Paray-Vieille-Poste - 91,Publiée le 22/01/2024,Actualisée le 22/01/2024,A négocier,Dès que possible,Minimum 2 ans,Développeur,Cadre du secteur privé,Régionale,CONSTRUCTION AÉRONAUTIQUE ET SPATIALE,Partiel possible,"Au sein du pôle Applications & Data, vous aure...",Diplômé d'un BAC +5 de type Ingénieur ou Maste...,[Anglais],"[Capacité d'analyse, Capacité de synthèse, Esp...","[Base de données, Microsoft SQL Server, Oracle...","Le Groupe ADP, un des leaders mondiaux de l'aé..."
498,Chambres d'Agriculture France,1 \n CDI,Caen - 14,Publiée le 18/01/2024,Actualisée le 18/01/2024,35 - 47 k€ brut annuel,Dès que possible,Minimum 2 ans,Chef de projet informatique,Cadre du secteur privé,Régionale,ACTIVITÉS DES ORGANISATIONS PATRONALES ET CONS...,Partiel possible,"Rattaché(e) au Pôle Data Urbanisation, vous tr...",De formation supérieure niveau Bac +5 (Master ...,[Aucune langue attendue],"[Autonomie, Capacité d’écoute, Esprit d'équipe...","[Apache Camel, Confluence, Docker, Framework, ...",Poste d'Ingénieur(e) Data à pourvoir à la Dire...
499,ORANGE BUSINESS,1 \n CDI,Rennes - 35,Publiée le 01/02/2024,Actualisée le 01/02/2024,A négocier,Dès que possible,Minimum 4 ans,Chief Data Officer,Cadre du secteur privé,Pas de déplacement,CONSEIL EN SYSTÈMES ET LOGICIELS INFORMATIQUES,Partiel possible,"Au sein de notre agence Rennaise, et intégré d...","De formation informatique Bac+5, vous justifie...",[Aucune langue attendue],"[Autonomie, Curiosité, Esprit d'équipe, Esprit...","[Extract Transform Load, Informatica, Informat...",L'ambition d'Orange Business est de devenir l'...
500,ACTUAL TOULOUSE INDUSTRIE,2 \n CDI,Blagnac - 31,Publiée le 02/02/2024,Actualisée le 02/02/2024,A partir de 40 k€ brut annuel,Dès que possible,Minimum 2 ans,Consultant technique progiciel,Cadre du secteur privé,Pas de déplacement,CONSEIL EN SYSTÈMES ET LOGICIELS INFORMATIQUES,Partiel possible,L'agence Actual recherche actuellement un Cons...,Actual recherche un consultant Microsoft Data ...,[Aucune langue attendue],"[Capacité à résoudre des problèmes, Capacité d...","[C sharp, Java, Langage de programmation, Pyth...","Véritable moteur dans son domaine, notre clien..."


## Création du fichier csv

In [193]:
df.to_csv(f'./data/Apec_data_{NB_JOBS}.csv', index=False)  # Assurez-vous de remplacer 'nom_du_fichier.csv' par le nom de fichier souhaité

## Toujours garder ce quit pour fermer la fenetre ouverte à la fin :

In [ ]:
#browser.quit()